In [26]:
valid_puzzles = [
  "00576224",
  # "009d5c81", hard
  "00dbd492",
  "03560426",
  # "05a7bcf2", skip
  "0607ce86",
  "0692e18c",
  "070dd51e",
  "08573cc6",
  "0934a4d8",
  # "09c534e7", # hard
  "0a1d4ef5",
  # "0a2355a6", # hard
  "0b17323b",
  "0bb8deee",
  "0becf7df", 
  "0c786b71",
  "0c9aba6e",
  "0d87d2a6",
  "0e671a1a",
  "0f63c0b9",
  "103eff5b",
  "11e1fe23", 
  "12422b43",
  "12997ef3", 
  "12eac192",
  # "136b0064", # hard
  "13713586",
  "137f0df0", 
  "140c817e", 
  "14754a24",
  # "15113be4", # hard
  # "15663ba9", # hard
  "15696249", 
  # "16b78196", # hard
  "17b80ad2", 
  "17cae0c1", 
  # "18419cfa", # hard
  "184a9768", 
  "195ba7dc", 
  # "1990f7a8",
  # "19bb5feb",
  # "1a2e2828",
  # "1a6449f1",
  # "1acc24af",
  # "1c02dbbe",
  # "1c0d0a4b",
  # "1c56ad9f",
  # "1d0a4b61",
  "1d398264",
]

print(len(valid_puzzles))

32


In [8]:
import os

def check_o1_html(puzzle_ids):
    results = {}
    for puzzle_id in puzzle_ids:
        file_path = os.path.join("html", puzzle_id, "o1.html")
        results[puzzle_id] = os.path.exists(file_path)
    return results

# Example usage
existence_results = check_o1_html(valid_puzzles)

for folder, exists in existence_results.items():
    print(f"o1.html {'exists' if exists else 'does not exist'} in {folder}")

o1.html exists in 00576224
o1.html exists in 00dbd492
o1.html exists in 03560426
o1.html exists in 0607ce86
o1.html exists in 0692e18c
o1.html exists in 070dd51e
o1.html exists in 08573cc6
o1.html exists in 0934a4d8
o1.html exists in 0a1d4ef5
o1.html exists in 0b17323b
o1.html exists in 0bb8deee
o1.html exists in 0becf7df
o1.html exists in 0c786b71
o1.html exists in 0c9aba6e
o1.html exists in 0d87d2a6
o1.html exists in 0e671a1a
o1.html exists in 0f63c0b9


In [11]:
import modal

fn = modal.Function.lookup("arc-generator", "generate_data_from_html")
for puzzle_id in valid_puzzles:
  fn_call = fn.spawn(puzzle_id, 100, 20, "html_dim_20_small")

In [28]:
import json
import os
import time
from arc_prize.synth_data.html import capture_html_screenshot, get_web_driver, process_screenshot

num_puzzles = 8000
dataset_dir = "/Users/pfh/work/arc-data/html"
edition = 1

os.makedirs(dataset_dir, exist_ok=True)

driver = get_web_driver(60, 150)

# puzzles = valid_puzzles
puzzles = ["1d398264"]

def validate_puzzle(puzzle: list[list[int]]) -> bool:
    if len(puzzle) < 2 or len(puzzle) > 5:
        return False
    for pair in puzzle:
        if len(pair) != 2:
            return False
    return True

for puzzle_id in puzzles:
    html_file = f"html/{puzzle_id}.html"
    current_dir = os.getcwd()
    full_path = os.path.join(current_dir, html_file)
    full_path = os.path.abspath(full_path)

    start_time = time.time()
    puzzles = []

    print("Starting", puzzle_id)

    for i in range(num_puzzles):
        try:
            raw_screenshot = capture_html_screenshot(driver, full_path)
            arc_puzzle_data, _ = process_screenshot(raw_screenshot)
        except Exception as e:
            print("Skipping", e)
            continue

        if validate_puzzle(arc_puzzle_data) is True:
            puzzles.append(arc_puzzle_data)

        if i % 100 == 0:
            duration = time.time() - start_time
            per_puzzle_time = duration / max(len(puzzles), 1)
            print(
                f"Iteration {i}, done {len(puzzles)} puzzles. Time elapsed: {duration:.2f}s ({(duration / 60):.2f}m). Per puzzle: {per_puzzle_time:.2f}s"
            )
    
    duration = time.time() - start_time
    print("Finished", puzzle_id, f"Total time: {duration:.2f}s ({(duration / 60):.2f}m)")

    with open(f"{dataset_dir}/{puzzle_id}_{edition}.json", "w") as f:
        json.dump(puzzles, f)

driver.quit()



Starting 1d398264
Iteration 0, done 1 puzzles. Time elapsed: 0.25s (0.00m). Per puzzle: 0.25s
Iteration 100, done 101 puzzles. Time elapsed: 19.45s (0.32m). Per puzzle: 0.19s
Iteration 200, done 201 puzzles. Time elapsed: 37.95s (0.63m). Per puzzle: 0.19s
Iteration 300, done 301 puzzles. Time elapsed: 56.33s (0.94m). Per puzzle: 0.19s
Iteration 400, done 401 puzzles. Time elapsed: 78.22s (1.30m). Per puzzle: 0.20s
Iteration 500, done 501 puzzles. Time elapsed: 97.19s (1.62m). Per puzzle: 0.19s
Iteration 600, done 601 puzzles. Time elapsed: 115.84s (1.93m). Per puzzle: 0.19s
Iteration 700, done 701 puzzles. Time elapsed: 135.30s (2.25m). Per puzzle: 0.19s
Iteration 800, done 801 puzzles. Time elapsed: 153.64s (2.56m). Per puzzle: 0.19s
Iteration 900, done 901 puzzles. Time elapsed: 172.07s (2.87m). Per puzzle: 0.19s
Iteration 1000, done 1001 puzzles. Time elapsed: 191.79s (3.20m). Per puzzle: 0.19s
Iteration 1100, done 1101 puzzles. Time elapsed: 210.20s (3.50m). Per puzzle: 0.19s
Itera

In [21]:
dataset_dir = "/Users/pfh/work/arc-data/html"

file_groups = {}

for filename in os.listdir(dataset_dir):
    if filename.endswith('.json'):
        prefix = filename.split('.')[0].split('_')[0]
        if file_groups.get(prefix, None) is None:
            file_groups[prefix] = []
        file_groups[prefix].append(filename)

# Process each group of files
for puzzle_id, files in file_groups.items():
    if len(files) == 1 and files[0] == f"{puzzle_id}.json":
        print(f"Skipping {puzzle_id} because there are only {len(files)} files")
        continue
    combined_data = []
    files_to_delete = []
    for file in files:
        with open(os.path.join(dataset_dir, file), 'r') as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    combined_data.extend(data)
                    files_to_delete.append(file)
                else:
                    print("Malformed file", file)
            except json.JSONDecodeError:
                print(f"Error decoding JSON in file: {file}")
    
    for file in files_to_delete:
        file_path = os.path.join(dataset_dir, file)
        os.remove(file_path)
        print(f"Deleted: {file}")

    # Write combined data to a new file
    output_filename = f"{puzzle_id}.json"
    with open(os.path.join(dataset_dir, output_filename), 'w') as f:
        json.dump(combined_data, f)
    
    print(f"Combined {len(files_to_delete)} files with {len(combined_data)} puzzles into {output_filename}")


Deleted: 137f0df0_1.json
Combined 1 files with 8000 puzzles into 137f0df0.json
Skipping 00dbd492 because there are only 1 files
Deleted: 12422b43_1.json
Deleted: 12422b43_2.json
Combined 2 files with 8000 puzzles into 12422b43.json
Skipping 103eff5b because there are only 1 files
Skipping 03560426 because there are only 1 files
Deleted: 12eac192_1.json
Combined 1 files with 8000 puzzles into 12eac192.json
Skipping 0c9aba6e because there are only 1 files
Deleted: 13713586_1.json
Combined 1 files with 8000 puzzles into 13713586.json
Skipping 0c786b71 because there are only 1 files
Skipping 11e1fe23 because there are only 1 files
Skipping 0a1d4ef5 because there are only 1 files
Skipping 0b17323b because there are only 1 files
Skipping 0e671a1a because there are only 1 files
Skipping 0692e18c because there are only 1 files
Skipping 0f63c0b9 because there are only 1 files
Skipping 0934a4d8 because there are only 1 files
Skipping 070dd51e because there are only 1 files
Deleted: 12997ef3_1.js

In [ ]:
import os
from arc_prize.synth_data.html import get_html, make_prompt

puzzle_id = "12422b43"
os.makedirs(f"html/{puzzle_id}", exist_ok=True)

print(make_prompt(puzzle_id))

# for model in ["claude"]:
#   for i in range(1):
#     response = get_html(puzzle_id, model, output_file_path=f"html/{puzzle_id}/{model}_{i}.html")
